In [1]:
import torch
import tiktoken
from GPT2Model import GPTModel, generate_text_simple, create_dataloader

In [2]:
GPT_CONFIG_124M = {
 "vocab_size": 50257,
 "context_length": 256, 
 "emb_dim": 768,
 "n_heads": 12,
 "n_layers": 12, 
 "drop_rate": 0.1, 
 "qkv_bias": False
}
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (Wq): Linear(in_features=768, out_features=768, bias=False)
        (Wk): Linear(in_features=768, out_features=768, bias=False)
        (Wv): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (Wq): Linear(in_features=768, out_features

In [3]:
# Some auxliary functions

def text_to_token_ids(txt, tokenizer):

    encoded = tokenizer.encode(txt, allowed_special = {'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # adds batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0)
    return tokenizer.decode(flat.tolist())



In [4]:
start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you rentingetic wasnم refres RexMeCHicular stren


In [5]:
with open("The-Verdict.txt", "r", encoding="utf-8") as f:
    text_data = f.read()

total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))
print("Characters:", total_characters)
print("Tokens:", total_tokens)

Characters: 20479
Tokens: 5145


In [6]:
train_ratio = 0.80
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

torch.manual_seed(123)
train_loader = create_dataloader(
 train_data,
 batch_size=4,
 max_length=GPT_CONFIG_124M["context_length"],
 stride=GPT_CONFIG_124M["context_length"],
 drop_last=True,
 shuffle=True,
 num_workers=0
)
val_loader = create_dataloader(
 val_data,
 batch_size=4,
 max_length=GPT_CONFIG_124M["context_length"],
 stride=GPT_CONFIG_124M["context_length"],
 drop_last=False,
 shuffle=False,
 num_workers=0
)


print("Train loader:")
for x, y in train_loader:
 print(x.shape, y.shape)
print("\nValidation loader:")
for x, y in val_loader:
 print(x.shape, y.shape)

Train loader:
torch.Size([4, 256]) torch.Size([4, 256])
torch.Size([4, 256]) torch.Size([4, 256])
torch.Size([4, 256]) torch.Size([4, 256])
torch.Size([4, 256]) torch.Size([4, 256])

Validation loader:
torch.Size([4, 256]) torch.Size([4, 256])


In [7]:
def calc_loss_batch(input_batch, target_batch, model, device):
    # computes loss for a single batch
    import torch.nn.functional as F
    input_batch = input_batch.to(device)
    target_batch = target_batch.to(device)
    logits = model(input_batch)
    loss = F.cross_entropy(logits.flatten(0,1), target_batch.flatten())

    return loss

def calc_loss_loader(data_loader, model, device, n_batches = None):
    total_loss = 0 # initialize
    if len(data_loader) == 0:
        return float('nan')
    elif n_batches is None:
        n_batches = len(data_loader)
    else:
        n_batches = min(n_batches, len(data_loader))
    
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < n_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss+=loss.item()
        else:
            break
    
    return total_loss / n_batches

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) 
with torch.no_grad(): 
 train_loss = calc_loss_loader(train_loader, model, device) 
 val_loss = calc_loss_loader(val_loader, model, device)
print("Training loss model not trained:", train_loss)
print("Validation loss model not trained:", val_loss)

Training loss model not trained: 10.992259740829468
Validation loss model not trained: 10.983443260192871


In [9]:
# Training an LLM

def train_model_simple(model, train_loader, val_loader, optimizer, 
                       device, n_epochs, eval_freq, eval_iter, start_context,
                       tokenizer):
    
    train_losses, val_losses, track_tokens_seen = [],[],[]
    tokens_seen, global_step = 0, -1

    # Training loop
    for epoch in range(n_epochs):
        model.train()
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() #reset gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # calculates gradients
            optimizer.step()
            tokens_seen += input_batch.numel()
            global_step += 1

            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Epoch {epoch+1} (Step {global_step:06d}): "
                      f"Train Loss {train_loss: .3f}, "
                      f"Val Loss {val_loss: .3f}"
                      )
    generate_and_print_sample(model, tokenizer, device, start_context)

    return train_losses, val_losses, track_tokens_seen

def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval() 
    with torch.no_grad(): 
        train_loss = calc_loss_loader(train_loader, model, device, n_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, n_batches=eval_iter)
    
    model.train()
    return train_loss, val_loss

def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
    decoded_text = token_ids_to_text(token_ids, tokenizer)
    print(decoded_text.replace("\n", " ")) 
    model.train()  

In [ ]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(
 model.parameters(), 
 lr=0.0004, weight_decay=0.1
)
n_epochs = 12
train_losses, val_losses, tokens_seen = train_model_simple(
 model, train_loader, val_loader, optimizer, device,
 n_epochs=n_epochs, eval_freq=5, eval_iter=5,
 start_context="Every effort moves you", tokenizer=tokenizer
)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
def plot_losses(epochs_seen, tokens_seen, train_losses, val_losses):
    fig, ax1 = plt.subplots(figsize=(5, 3))
    ax1.plot(epochs_seen, train_losses, label="Training loss")
    ax1.plot(
    epochs_seen, val_losses, linestyle="-.", label="Validation loss"
    )
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.legend(loc="upper right")
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax2 = ax1.twiny() 
    ax2.plot(tokens_seen, train_losses, alpha=0) 
    ax2.set_xlabel("Tokens seen")
    fig.tight_layout()
    plt.show()
    
epochs_tensor = torch.linspace(0, n_epochs, len(train_losses))
plot_losses(epochs_tensor, tokens_seen, train_losses, val_losses)

In [10]:
def generate_text(model, idx, max_new_tokens, context_size, temp = 0.0, top_k = None, eos_id = None):

    ''' 
    This function is an evolution to take into account randomness with the temperature, and top results
    from sampling probabilities with top_k
    
    '''
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        
        logits = logits[:, -1, :] # Only tthe last time step, so that (batch, n_token, vocab_size) becomes (batch, vocab_size)

        # top_k choices
        if top_k is not None:
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(
                logits < min_val, 
                torch.tensor(float('-inf')).to(logits.device),
                logits
            )
            
        if temp > 0.0:
            logits = logits / temp
            probas = torch.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probas, num_samples = 1)
        else:
            idx_next = torch.argmax(probas, dim = -1, keepdim= True)
        

        idx = torch.cat((idx, idx_next), dim = -1)
    
    return idx

In [ ]:
torch.manual_seed(123)
token_ids = generate_text(
 model=model,
 idx=text_to_token_ids("Every effort moves you", tokenizer),
 max_new_tokens=15,
 context_size=GPT_CONFIG_124M["context_length"],
 top_k=25,
 temp=1.4
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

In [27]:
# Loading and saving model in PyTorch
torch.save(model.state_dict(), "model.pth")

In [ ]:
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(torch.load("model.pth", map_location=device))
model.eval()

In [29]:
torch.save({
 "model_state_dict": model.state_dict(),
 "optimizer_state_dict": optimizer.state_dict(),
 }, 
 "model_and_optimizer.pth"
)

In [ ]:
checkpoint = torch.load("model_and_optimizer.pth", map_location=device)
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(checkpoint["model_state_dict"])
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.1)
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
model.train()

In [ ]:
import urllib.request
url = (
 "https://raw.githubusercontent.com/rasbt/"
 "LLMs-from-scratch/main/ch05/"
 "01_main-chapter-code/gpt_download.py"
)
filename = url.split('/')[-1]
urllib.request.urlretrieve(url, filename)

In [35]:
# The tensorflow download doesn´t work with pip in this local notebook, let´s try with the transformers library
# as Pytorch weights are available as a model.safetensors file

import numpy as np


# allowed model names
model_names = {
    "gpt2-small (124M)": "openai-community/gpt2",
    "gpt2-medium (355M)": "openai-community/gpt2-medium",
    "gpt2-large (774M)": "openai-community/gpt2-large",
    "gpt2-xl (1558M)": "openai-community/gpt2-xl"
}

CHOOSE_MODEL = "gpt2-small (124M)"

BASE_CONFIG = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "drop_rate": 0.0,       # Dropout rate
    "qkv_bias": True        # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}


BASE_CONFIG.update(model_configs[CHOOSE_MODEL])
BASE_CONFIG

{'vocab_size': 50257,
 'context_length': 1024,
 'drop_rate': 0.0,
 'qkv_bias': True,
 'emb_dim': 768,
 'n_layers': 12,
 'n_heads': 12}

In [74]:
def assign_check(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch. Left: {left.shape}, Right: {right.shape}")
    return torch.nn.Parameter(right.clone().detach())

def load_weights(gpt, gpt_hf):

    #d = gpt_hf.state_dict()
    d = gpt_hf # If we load the model.safetensors is loaded already as a dict
    
    gpt.pos_emb.weight = assign_check(gpt.pos_emb.weight, d["wpe.weight"])
    gpt.tok_emb.weight = assign_check(gpt.tok_emb.weight, d["wte.weight"])
    
    for b in range(BASE_CONFIG["n_layers"]):
        q_w, k_w, v_w = np.split(d[f"h.{b}.attn.c_attn.weight"], 3, axis=-1)
        gpt.trf_blocks[b].att.Wq.weight = assign_check(gpt.trf_blocks[b].att.Wq.weight, q_w.T)
        gpt.trf_blocks[b].att.Wk.weight = assign_check(gpt.trf_blocks[b].att.Wk.weight, k_w.T)
        gpt.trf_blocks[b].att.Wv.weight = assign_check(gpt.trf_blocks[b].att.Wv.weight, v_w.T)
    
        q_b, k_b, v_b = np.split(d[f"h.{b}.attn.c_attn.bias"], 3, axis=-1)
        gpt.trf_blocks[b].att.Wq.bias = assign_check(gpt.trf_blocks[b].att.Wq.bias, q_b)
        gpt.trf_blocks[b].att.Wk.bias = assign_check(gpt.trf_blocks[b].att.Wk.bias, k_b)
        gpt.trf_blocks[b].att.Wv.bias = assign_check(gpt.trf_blocks[b].att.Wv.bias, v_b)
    
    
        gpt.trf_blocks[b].att.out_proj.weight = assign_check(gpt.trf_blocks[b].att.out_proj.weight, d[f"h.{b}.attn.c_proj.weight"].T)
        gpt.trf_blocks[b].att.out_proj.bias = assign_check(gpt.trf_blocks[b].att.out_proj.bias, d[f"h.{b}.attn.c_proj.bias"])
    
        gpt.trf_blocks[b].ff.layers[0].weight = assign_check(gpt.trf_blocks[b].ff.layers[0].weight, d[f"h.{b}.mlp.c_fc.weight"].T)
        gpt.trf_blocks[b].ff.layers[0].bias = assign_check(gpt.trf_blocks[b].ff.layers[0].bias, d[f"h.{b}.mlp.c_fc.bias"])
        gpt.trf_blocks[b].ff.layers[2].weight = assign_check(gpt.trf_blocks[b].ff.layers[2].weight, d[f"h.{b}.mlp.c_proj.weight"].T)
        gpt.trf_blocks[b].ff.layers[2].bias = assign_check(gpt.trf_blocks[b].ff.layers[2].bias, d[f"h.{b}.mlp.c_proj.bias"])
    
        gpt.trf_blocks[b].norm1.scale = assign_check(gpt.trf_blocks[b].norm1.scale, d[f"h.{b}.ln_1.weight"])
        gpt.trf_blocks[b].norm1.shift = assign_check(gpt.trf_blocks[b].norm1.shift, d[f"h.{b}.ln_1.bias"])
        gpt.trf_blocks[b].norm2.scale = assign_check(gpt.trf_blocks[b].norm2.scale, d[f"h.{b}.ln_2.weight"])
        gpt.trf_blocks[b].norm2.shift = assign_check(gpt.trf_blocks[b].norm2.shift, d[f"h.{b}.ln_2.bias"])
    
        gpt.final_norm.scale = assign_check(gpt.final_norm.scale, d[f"ln_f.weight"])
        gpt.final_norm.shift = assign_check(gpt.final_norm.shift, d[f"ln_f.bias"])
        gpt.out_head.weight = assign_check(gpt.out_head.weight, d["wte.weight"])

In [1]:
import torch
from safetensors.torch import load
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
file_path = "model.safetensors"
with open(file_path, "rb") as f:
    data = f.read()

loaded_gpt = load(data)
loaded_gpt

{'h.10.attn.c_attn.bias': tensor([-0.0301,  0.1360, -0.3842,  ..., -0.0599,  0.1059,  0.0276]),
 'h.6.attn.c_attn.bias': tensor([ 0.0380,  0.1714, -0.1409,  ..., -0.0441,  0.0544,  0.0041]),
 'h.4.ln_2.weight': tensor([0.2568, 0.2607, 0.2783, 0.2811, 0.2852, 0.2770, 0.2750, 0.1547, 0.2835,
         0.2898, 0.2725, 0.2783, 0.2861, 0.2720, 0.2651, 0.2725, 0.2639, 0.2858,
         0.2607, 0.2527, 0.2881, 0.2705, 0.2900, 0.2565, 0.2698, 0.2764, 0.2840,
         0.2685, 0.2937, 0.2858, 0.2844, 0.2510, 0.2921, 0.2729, 0.2841, 0.2604,
         0.2445, 0.2936, 0.2588, 0.2803, 0.2760, 0.3091, 0.2919, 0.2764, 0.2627,
         0.2627, 0.2666, 0.2882, 0.2860, 0.2790, 0.2627, 0.2783, 0.2588, 0.2801,
         0.2967, 0.2735, 0.2830, 0.2638, 0.2750, 0.2461, 0.2649, 0.2702, 0.2413,
         0.2473, 0.1929, 0.3029, 0.2939, 0.2955, 0.2293, 0.2784, 0.2837, 0.1469,
         0.2803, 0.2959, 0.2978, 0.2647, 0.2939, 0.2627, 0.2807, 0.2681, 0.2972,
         0.2960, 0.2978, 0.2764, 0.2495, 0.2530, 0.2565, 0.07

In [77]:
from safetensors.torch import load
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
file_path = "model.safetensors"
with open(file_path, "rb") as f:
    data = f.read()

loaded_gpt = load(data)

gpt = GPTModel(BASE_CONFIG)

load_weights(gpt, loaded_gpt)

In [78]:
torch.manual_seed(123)

tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text(
    model=gpt.to(device),
    idx=text_to_token_ids("Every effort moves", tokenizer).to(device),
    max_new_tokens=30,
    context_size=BASE_CONFIG["context_length"],
    top_k=1,
    temp=1.0
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves forward, but it's not enough.

"I'm not going to sit here and say, 'I'm not going to do this,'
